In [8]:

"""
# Author: Zhongyi Zhang
# Title: BCBS Data Science Engineer Coding Prompt Answer Main Program (Available to run in terminal passing input and output path_to_file.txt)

# type in input file path and output file path pass as arguments in the command line
# so that I can directly run the below command in terminal
# first cd to the path where this py file locate, then run below command in terminal:
# $ python zhongyi_zhang_bcbs_answer.py <"path_to_billing_log.txt"> <"path_to_output_cost_report.txt">
# Then go to the output file path and I will get the output cost_report.txt file
"""

# from argparse import ArgumentParser
from collections import deque
from datetime import datetime

class Billing:
    def __init__(self):
        """
        # initialize parameters, create input and output path as arguments in the command line
        """
#         parser = ArgumentParser(description="Generate cost report txt file.")
#         parser.add_argument('input_path_file', metavar = 'input_path_file', type=str, help='input txt file path')
#         parser.add_argument('output_path_file', metavar = 'output_path_file', type=str, help='output txt file path')

#         args = parser.parse_args()
#         self.input_path_file = args.input_path_file
#         self.output_path_file = args.output_path_file
        
        self.all_data = all_data = []
        self.order = order = []
        self.bills = bills = []
        self.members_providers = members_providers = []
        self.d = d = {}
    
    """ 
    # input and output path and file ready in string format; algorithms start from here:
    """ 
    def reader(self):
        """ 
        # open the input txt file, read in lines, and convert into a list with removing leading and trailing white spaces
        """ 
        with open('billing_log_small.txt') as w:
            self.all_data = [item.lstrip().rstrip() for item in w.readlines()]
        return self.all_data
    
    def extract_declared_orders(self, all_data):
        """ 
        # create a list to record the declared order for members and healthcare providers. Members first, then providers
        """
        members_list = []
        providers_list = []
        for i in self.all_data:
            if str(i[0:6]).upper() == 'MEMBER' and i[7:] not in members_list:
                members_list.append("* " + i[7:])
            elif str(i[0:8]).upper() == 'PROVIDER' and i[9:] not in providers_list:
                providers_list.append("* " + i[9:])

        self.order = members_list + providers_list
        return self.order
    
    def extract_bills(self, all_data):
        """
        # filter for all the bills record from the input billing log txt file and convert into a list array
        # break the all_bills list into nested list to separate members/providers name, date, and amount
        # here I use the deque open-source data structure since I can directly manipulate data from both ends of the list
        # I also applied stack algorithm techniques to always append the popped item from the stack (last-in-first-out)
        
        # no need to have the word "Bill" as the first element
        # only members name, date, and amount 
        # the rest in the stack is the healthcare providers name. convert back to string
        """
        all_bills = [i for i in self.all_data if str(i[0:4]).upper() == 'BILL']

        for i, j in enumerate(all_bills):
            self.bills.append(deque())
            stack = j.split(' ')[1:]    # no need to have the word "Bill"
            for k in j.split(' ')[-1:-4:-1]:    # only members name, date, and amount 
                self.bills[i].appendleft(stack.pop()) 
            self.bills[i].appendleft(' '.join(stack))    # the rest in the stack is the healthcare providers name. convert back to string
        self.bills = [list(i) for i in self.bills]    # convert back to list from deque

        """
        # convert the amount into float datatype 
        """
        for i in self.bills: 
            i[-1] = float(i[-1])
            
        """
        # create a function to convert date into month format "YYYY-mm-01"
        # appending a new letter-format month year element at the end of each bill
        """
        def convert_month_year(l: list):
            for i in l:
                i[-2] = i[-2][0:8] + '01'
                if i[-2][5:7] == '01':
                    i.append('January ' + i[-2][:4])
                elif i[-2][5:7] == '02':
                    i.append('February ' + i[-2][:4])
                elif i[-2][5:7] == '03':
                    i.append('March ' + i[-2][:4])
                elif i[-2][5:7] == '04':
                    i.append('April ' + i[-2][:4])
                elif i[-2][5:7] == '05':
                    i.append('May ' + i[-2][:4])
                elif i[-2][5:7] == '06':
                    i.append('June ' + i[-2][:4])
                elif i[-2][5:7] == '07':
                    i.append('July ' + i[-2][:4])
                elif i[-2][5:7] == '08':
                    i.append('August ' + i[-2][:4])
                elif i[-2][5:7] == '09':
                    i.append('September ' + i[-2][:4])
                elif i[-2][5:7] == '10':
                    i.append('October ' + i[-2][:4])
                elif i[-2][5:7] == '11':
                    i.append('November ' + i[-2][:4])
                elif i[-2][5:7] == '12':
                    i.append('December ' + i[-2][:4])
            return l

        convert_month_year(self.bills) # apply the above function to the list
        
        """
        # convert into datetime datatype and sort by month first for later data preparation
        """
        self.bills.sort(key=lambda x: datetime.strptime(x[2], "%Y-%m-%d"))
        return self.bills
    
    def data_cleaning(self, bills):
        """
        # combine members and providers name into one list. re-organize the list and filter only the data I will need later
        # I will only need the letter-format of month year, members/providers name, and amount
        # adding a "* " in front of members and providers name
        """
        providers = []
        members = []
        for i, j in enumerate(self.bills):
            members.append([])
            providers.append([])
            members[i].append(j[-1] + ":")
            members[i].append("* " + j[1]) # adding a "* " in front of members name
            members[i].append(j[-2])
            providers[i].append(j[-1] + ":")
            providers[i].append("* " + j[0]) # adding a "* " in front of healthcare providers name
            providers[i].append(j[-2])
        self.members_providers = members + providers  
        
        return self.members_providers
    
    def calculating_bills(self, members_providers):
        """
        # This method is to create a dictionary to sum the amount group by month-year and member/provider name
        # first create a nested dictionary with keys as month-year
        # members and providers name will be the keys of sub-dictionary, values will be the accumulated amount per member/provider per month-year
        # if month-year in the dictionary but members/providers name not in the sub-dictionary, create the key value pair with the amount
        # if month-year in the dictionary and members/providers name also in the sub-dictionary, adding/accumulate the amount    
        """
        self.d = {i[0]: dict() for i in self.members_providers} 
        for i in self.members_providers:
            if i[0] not in self.d.keys():  # this condition shouldn't be used since I already created all the month-year as keys in the dictionary. just in case
                self.d.update({i[0]: dict()})
            if i[0] in self.d.keys() and i[1] not in self.d[i[0]].keys():
                self.d[i[0]].update({i[1]: i[2]})
            elif i[0] in self.d.keys() and i[1] in self.d[i[0]].keys():  
                self.d[i[0]][i[1]] += i[2]
        return self.d
    
    def formatting(self, d, order):
        """
        # the above dictionary is in the Bills order for members/providers name,
        # so I need to change to the "declared order" to match with the declared order list I created at beginning
        """
        for i, j in self.d.items():
            self.d[i] = {m: n for m, n in sorted(j.items(), key=lambda pair: self.order.index(pair[0]))}
        
        """
        # for each amount value, adding "$" dollar sign in the front, separated by comma, and round to 2 decimal places
        """
        for i, j in self.d.items():
            for k, v in j.items():
                j[k] = "$" + "{0:,.2f}".format(v)
        
        return self.d
    
    def writer(self, d):
        """
        # To write the above nested dictionary into a cost report txt file with the output path from the terminal command argument
        """
        with open('cost_report_zhongyi_zhang.txt', 'w') as f:
            for month_year, name in self.d.items():
                f.write(month_year + '\n')
                f.write("\n".join(["{}: {}".format(value_key, digit) for value_key, digit in name.items()]))
                f.write('\n')



billing_logs = Billing()
data = billing_logs.reader()
declared_order = billing_logs.extract_declared_orders(data)
bill = billing_logs.extract_bills(data)
cleaned_data = billing_logs.data_cleaning(bill)
amount_hashtable = billing_logs.calculating_bills(cleaned_data)
final_data = billing_logs.formatting(amount_hashtable, declared_order)
billing_logs.writer(final_data)    

                
                
"""
# testing the file in the command
# run above methods from the class in order
"""
# if __name__ == '__main__':
#     try:
#         billing_logs = Billing()
#         data = billing_logs.reader()
#         declared_order = billing_logs.extract_declared_orders(data)
#         bill = billing_logs.extract_bills(data)
#         cleaned_data = billing_logs.data_cleaning(bill)
#         amount_hashtable = billing_logs.calculating_bills(cleaned_data)
#         final_data = billing_logs.formatting(amount_hashtable, declared_order)
#         billing_logs.writer(final_data)
#         print("Finished. Please check your output path, and the cost report.txt file will be there.")
#     except FileNotFoundError:
#         print("Please double check your file path. The path is not found.")



'\n# testing the file in the command\n# run above methods from the class in order\n'

In [3]:
with open('billing_log_small.txt') as w:
    all_data = [item.lstrip().rstrip() for item in w.readlines()]
members_list = []
providers_list = []
for i in all_data:
    if str(i[0:6]).upper() == 'MEMBER' and i[7:] not in members_list:
        members_list.append("* " + i[7:])
    elif str(i[0:8]).upper() == 'PROVIDER' and i[9:] not in providers_list:
        providers_list.append("* " + i[9:])

order = members_list + providers_list
print(order)

['* Edgar', '* Frannie', '* Michael', '* EastShore Healthcare', '* Alliance Medical Group']


In [23]:
all_data

['Member Edgar',
 'Member Frannie',
 'Provider EastShore Healthcare',
 'Provider Alliance Medical Group',
 'Bill Alliance Medical Group Frannie 2021-02-02 1050.00',
 'Member Michael',
 'Bill Alliance Medical Group Edgar 2021-01-24 299.25',
 'Bill EastShore Healthcare Michael 2021-01-01 6014.30',
 'Bill EastShore Healthcare Edgar 2021-04-12 3000.00',
 'Bill EastShore Healthcare Edgar 2021-01-07 499.25']

In [24]:
from collections import deque
all_bills = [i for i in all_data if str(i[0:4]).upper() == 'BILL']
print(all_bills, "\n"*5)

bills = []
for i, j in enumerate(all_bills):
    bills.append(deque())
    stack = j.split(' ')[1:]    # no need to have the word "Bill"
    for k in j.split(' ')[-1:-4:-1]:    # only members name, date, and amount 
        bills[i].appendleft(stack.pop()) 
    bills[i].appendleft(' '.join(stack))    # the rest in the stack is the healthcare providers name. convert back to string
bills = [list(i) for i in bills] 
print(bills)


['Bill Alliance Medical Group Frannie 2021-02-02 1050.00', 'Bill Alliance Medical Group Edgar 2021-01-24 299.25', 'Bill EastShore Healthcare Michael 2021-01-01 6014.30', 'Bill EastShore Healthcare Edgar 2021-04-12 3000.00', 'Bill EastShore Healthcare Edgar 2021-01-07 499.25'] 





[['Alliance Medical Group', 'Frannie', '2021-02-02', '1050.00'], ['Alliance Medical Group', 'Edgar', '2021-01-24', '299.25'], ['EastShore Healthcare', 'Michael', '2021-01-01', '6014.30'], ['EastShore Healthcare', 'Edgar', '2021-04-12', '3000.00'], ['EastShore Healthcare', 'Edgar', '2021-01-07', '499.25']]


In [25]:
all_bills = [i for i in all_data if str(i[0:4]).upper() == 'BILL']

bills = []
for i, j in enumerate(all_bills):
    bills.append(list())
    stack = j.split(' ')[1:]    # no need to have the word "Bill"
    for k in j.split(' ')[-1:-4:-1]:    # only members name, date, and amount 
        bills[i].append(stack.pop()) 
    bills[i].append(' '.join(stack))    # the rest in the stack is the healthcare providers name. convert back to string
bills = [i[::-1] for i in bills]
print(bills)


[['Alliance Medical Group', 'Frannie', '2021-02-02', '1050.00'], ['Alliance Medical Group', 'Edgar', '2021-01-24', '299.25'], ['EastShore Healthcare', 'Michael', '2021-01-01', '6014.30'], ['EastShore Healthcare', 'Edgar', '2021-04-12', '3000.00'], ['EastShore Healthcare', 'Edgar', '2021-01-07', '499.25']]


In [26]:
for i in bills: 
    i[-1] = float(i[-1])
        
from datetime import datetime
def convert_month_year(l: list):
    for i in l:
        i[-2] = i[-2][0:8] + '01'
        if i[-2][5:7] == '01':
            i.append('January ' + i[-2][:4])
        elif i[-2][5:7] == '02':
            i.append('February ' + i[-2][:4])
        elif i[-2][5:7] == '03':
            i.append('March ' + i[-2][:4])
        elif i[-2][5:7] == '04':
            i.append('April ' + i[-2][:4])
        elif i[-2][5:7] == '05':
            i.append('May ' + i[-2][:4])
        elif i[-2][5:7] == '06':
            i.append('June ' + i[-2][:4])
        elif i[-2][5:7] == '07':
            i.append('July ' + i[-2][:4])
        elif i[-2][5:7] == '08':
            i.append('August ' + i[-2][:4])
        elif i[-2][5:7] == '09':
            i.append('September ' + i[-2][:4])
        elif i[-2][5:7] == '10':
            i.append('October ' + i[-2][:4])
        elif i[-2][5:7] == '11':
            i.append('November ' + i[-2][:4])
        elif i[-2][5:7] == '12':
            i.append('December ' + i[-2][:4])
    return l

convert_month_year(bills) # apply the above function to the list

"""
# convert into datetime datatype and sort by month first for later data preparation
"""
bills.sort(key=lambda x: datetime.strptime(x[2], "%Y-%m-%d"))
print(bills)

[['Alliance Medical Group', 'Edgar', '2021-01-01', 299.25, 'January 2021'], ['EastShore Healthcare', 'Michael', '2021-01-01', 6014.3, 'January 2021'], ['EastShore Healthcare', 'Edgar', '2021-01-01', 499.25, 'January 2021'], ['Alliance Medical Group', 'Frannie', '2021-02-01', 1050.0, 'February 2021'], ['EastShore Healthcare', 'Edgar', '2021-04-01', 3000.0, 'April 2021']]


In [27]:
providers = []
members = []
for i, j in enumerate(bills):
    members.append([])
    providers.append([])
    members[i].append(j[-1] + ":")
    members[i].append("* " + j[1]) # adding a "* " in front of members name
    members[i].append(j[-2])
    providers[i].append(j[-1] + ":")
    providers[i].append("* " + j[0]) # adding a "* " in front of healthcare providers name
    providers[i].append(j[-2])
members_providers = members + providers  

print(members_providers)

[['January 2021:', '* Edgar', 299.25], ['January 2021:', '* Michael', 6014.3], ['January 2021:', '* Edgar', 499.25], ['February 2021:', '* Frannie', 1050.0], ['April 2021:', '* Edgar', 3000.0], ['January 2021:', '* Alliance Medical Group', 299.25], ['January 2021:', '* EastShore Healthcare', 6014.3], ['January 2021:', '* EastShore Healthcare', 499.25], ['February 2021:', '* Alliance Medical Group', 1050.0], ['April 2021:', '* EastShore Healthcare', 3000.0]]


In [28]:
d = {i[0]: dict() for i in members_providers} 
print(d, "\n"*3)

for i in members_providers:
    if i[0] not in d.keys():  # this condition shouldn't be used since I already created all the month-year as keys in the dictionary. just in case
        d.update({i[0]: dict()})
    if i[0] in d.keys() and i[1] not in d[i[0]].keys():
        d[i[0]].update({i[1]: i[2]})
    elif i[0] in d.keys() and i[1] in d[i[0]].keys():  
        d[i[0]][i[1]] += i[2]
print(d)


{'January 2021:': {}, 'February 2021:': {}, 'April 2021:': {}} 



{'January 2021:': {'* Edgar': 798.5, '* Michael': 6014.3, '* Alliance Medical Group': 299.25, '* EastShore Healthcare': 6513.55}, 'February 2021:': {'* Frannie': 1050.0, '* Alliance Medical Group': 1050.0}, 'April 2021:': {'* Edgar': 3000.0, '* EastShore Healthcare': 3000.0}}


In [41]:
dd = {}
for i in members_providers:
    dd.update({i[0]: dict()})
dd.update({'February 2021:': {}})
dd

{'January 2021:': {}, 'February 2021:': {}, 'April 2021:': {}}

In [50]:
order

['* Edgar',
 '* Frannie',
 '* Michael',
 '* EastShore Healthcare',
 '* Alliance Medical Group']

In [51]:
order2 = [i[2:] for i in order]
order2

['Edgar',
 'Frannie',
 'Michael',
 'EastShore Healthcare',
 'Alliance Medical Group']

In [54]:
for i, j in d.items():
    d[i] = {m: n for m, n in sorted(j.items(), key=lambda pair: order.index(pair[0]))}
d

{'January 2021:': {'* Edgar': 798.5,
  '* Michael': 6014.3,
  '* EastShore Healthcare': 6513.55,
  '* Alliance Medical Group': 299.25},
 'February 2021:': {'* Frannie': 1050.0, '* Alliance Medical Group': 1050.0},
 'April 2021:': {'* Edgar': 3000.0, '* EastShore Healthcare': 3000.0}}

In [56]:
d

{'January 2021:': {'* Edgar': 798.5,
  '* Michael': 6014.3,
  '* EastShore Healthcare': 6513.55,
  '* Alliance Medical Group': 299.25},
 'February 2021:': {'* Frannie': 1050.0, '* Alliance Medical Group': 1050.0},
 'April 2021:': {'* Edgar': 3000.0, '* EastShore Healthcare': 3000.0}}

In [57]:
{k: {k:"$" + "{0:,.2f}".format(v) for k,v in v.items()} for k,v in d.items()}

{'January 2021:': {'* Edgar': '$798.50',
  '* Michael': '$6,014.30',
  '* EastShore Healthcare': '$6,513.55',
  '* Alliance Medical Group': '$299.25'},
 'February 2021:': {'* Frannie': '$1,050.00',
  '* Alliance Medical Group': '$1,050.00'},
 'April 2021:': {'* Edgar': '$3,000.00',
  '* EastShore Healthcare': '$3,000.00'}}

In [60]:
for i, j in d.items():
    for k, v in j.items():
        j[k] = "$" + "{0:,.2f}".format(v)

print(d)

{'January 2021:': {'* Edgar': '$798.50', '* Michael': '$6,014.30', '* EastShore Healthcare': '$6,513.55', '* Alliance Medical Group': '$299.25'}, 'February 2021:': {'* Frannie': '$1,050.00', '* Alliance Medical Group': '$1,050.00'}, 'April 2021:': {'* Edgar': '$3,000.00', '* EastShore Healthcare': '$3,000.00'}}


In [64]:
for i, j in d.items():
    print(j)

{'* Edgar': 798.5, '* Michael': 6014.3, '* Alliance Medical Group': 299.25, '* EastShore Healthcare': 6513.55}
{'* Frannie': 1050.0, '* Alliance Medical Group': 1050.0}
{'* Edgar': 3000.0, '* EastShore Healthcare': 3000.0}


In [65]:
print(j)

{'* Edgar': 3000.0, '* EastShore Healthcare': 3000.0}


In [69]:
for k, v in j.items():
    print(k,'ooooo', v)

* Edgar ooooo 3000.0
* EastShore Healthcare ooooo 3000.0


In [80]:
"{1:.2f}". format(345.7949999,568.2024)

'568.20'

In [81]:
txt = "For only {price:.2f} dollars!"
print(txt.format(price = 49))

For only 49.00 dollars!


In [88]:
j = {'* Edgar': 3000.0, '* EastShore Healthcare': 3000.0}
for k,v in j.items():
    print(k,v)
    j[k] = "$" + "{0:,.2f}".format(v)
    break
#     print('\n')
j

* Edgar 3000.0


{'* Edgar': '$3,000.00', '* EastShore Healthcare': 3000.0}

In [120]:
d

{'January 2021:': {'* Edgar': 798.5,
  '* Michael': 6014.3,
  '* Alliance Medical Group': 299.25,
  '* EastShore Healthcare': 6513.55},
 'February 2021:': {'* Frannie': 1050.0, '* Alliance Medical Group': 1050.0},
 'April 2021:': {'* Edgar': 3000.0, '* EastShore Healthcare': 3000.0}}

In [18]:
{i:j for k,("$" + "{0:,.2f}".format(v)) in j.items() for i,j in d.items()}

SyntaxError: cannot assign to operator (<ipython-input-18-b30867ec48c3>, line 1)

In [34]:
{k: {k: "$" + "{0:,.2f}".format(v) for k, v in v.items()} for k, v in d.items()}

{'January 2021:': {'* Edgar': '$798.50',
  '* Michael': '$6,014.30',
  '* Alliance Medical Group': '$299.25',
  '* EastShore Healthcare': '$6,513.55'},
 'February 2021:': {'* Frannie': '$1,050.00',
  '* Alliance Medical Group': '$1,050.00'},
 'April 2021:': {'* Edgar': '$3,000.00',
  '* EastShore Healthcare': '$3,000.00'}}

In [33]:
dd

{'January 2021:': {'* Edgar': '$798.50',
  '* Michael': '$6,014.30',
  '* Alliance Medical Group': '$299.25',
  '* EastShore Healthcare': '$6,513.55'},
 'February 2021:': {'* Frannie': '$1,050.00',
  '* Alliance Medical Group': '$1,050.00'},
 'April 2021:': {'* Edgar': '$3,000.00',
  '* EastShore Healthcare': '$3,000.00'}}

In [105]:
for i,j in d.items():
    print(i,j)


January 2021: {'* Edgar': 798.5, '* Michael': 6014.3, '* Alliance Medical Group': 299.25, '* EastShore Healthcare': 6513.55}
February 2021: {'* Frannie': 1050.0, '* Alliance Medical Group': 1050.0}
April 2021: {'* Edgar': 3000.0, '* EastShore Healthcare': 3000.0}


In [122]:
d

{'January 2021:': {'* Edgar': 798.5,
  '* Michael': 6014.3,
  '* Alliance Medical Group': 299.25,
  '* EastShore Healthcare': 6513.55},
 'February 2021:': {'* Frannie': 1050.0, '* Alliance Medical Group': 1050.0},
 'April 2021:': {'* Edgar': 3000.0, '* EastShore Healthcare': 3000.0}}

In [124]:
{a:b for i,j in b.items() for a,b in d.items()}

{'January 2021:': {'* Edgar': 798.5,
  '* Michael': 6014.3,
  '* Alliance Medical Group': 299.25,
  '* EastShore Healthcare': 6513.55},
 'February 2021:': {'* Frannie': 1050.0, '* Alliance Medical Group': 1050.0},
 'April 2021:': {'* Edgar': 3000.0, '* EastShore Healthcare': 3000.0}}

In [116]:
ddd = {'category 1:': 
         {'A': 50.5,
          'B': 28.9,
          'C': 46.3,
          'D': 513.8},
       'category 2:': {'E': 20.5, 'C': 48.1},
       'category 3:': {'D': 28.2, 'F': 68.3}}
# for i,j in ddd.items():
#     for k,v in j.items():
#         j[k] = "$" + "{0:,.2f}".format(v)
for a,b in ddd.items():
    for i,j in b.items():
        b[i] = "$" + "{0:,.2f}".format(j)
ddd

{'category 1:': {'A': '$50.50', 'B': '$28.90', 'C': '$46.30', 'D': '$513.80'},
 'category 2:': {'E': '$20.50', 'C': '$48.10'},
 'category 3:': {'D': '$28.20', 'F': '$68.30'}}

In [123]:
ddd = {'category 1:': 
         {'A': 50.5,
          'B': 28.9,
          'C': 46.3,
          'D': 513.8},
       'category 2:': {'E': 20.5, 'C': 48.1},
       'category 3:': {'D': 28.2, 'F': 68.3}}
{a:b for i,j in b.items() for a,b in ddd.items()}

{'category 1:': {'A': 50.5, 'B': 28.9, 'C': 46.3, 'D': 513.8},
 'category 2:': {'E': 20.5, 'C': 48.1},
 'category 3:': {'D': 28.2, 'F': 68.3}}

In [118]:
{i:j for a,b in ddd.items() for i,j in b.items()}

{'A': '$50.50',
 'B': '$28.90',
 'C': '$48.10',
 'D': '$28.20',
 'E': '$20.50',
 'F': '$68.30'}

In [ ]:
{'category 1:': {'A': '$50.50', 'B': '$28.90', 'C': '$46.30', 'D': '$513.80'},
 'category 2:': {'E': '$20.50', 'C': '$48.10'},
 'category 3:': {'D': '$28.20', 'F': '$68.30'}}

In [64]:
import argparse

parser = argparse.ArgumentParser(description='Process some integers.')
parser.add_argument('integers', metavar='N', type=int, nargs='+',
                    help='an integer for the accumulator')
parser.add_argument('--sum', dest='accumulate', action='store_const',
                    const=sum, default=max,
                    help='sum the integers (default: find the max)')

args = parser.parse_args()
print(args.accumulate(args.integers))

usage: ipykernel_launcher.py [-h] [--sum] N [N ...]
ipykernel_launcher.py: error: argument N: invalid int value: '/Users/zhongyizhang/Library/Jupyter/runtime/kernel-0a4ff281-99a3-4338-bb74-e1af94691590.json'
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/zhongyizhang/opt/anaconda3/lib/python3.8/argparse.py", line 2422, in _get_value
    result = type_func(arg_string)
ValueError: invalid literal for int() with base 10: '/Users/zhongyizhang/Library/Jupyter/runtime/kernel-0a4ff281-99a3-4338-bb74-e1af94691590.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/zhongyizhang/opt/anaconda3/lib/python3.8/argparse.py", line 1800, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/Users/zhongyizhang/opt/anaconda3/lib/python3.8/argparse.py", line 2009, in _parse_known_args
    stop_index = consume_positionals(start_index)
  File "/Users/zhongyizhang/opt/anaconda3/lib/python3.8/argparse.py", line 1965, in consume_positionals
    take_action(action, args)
  File "/Users/zhongyizhang/opt/anaconda3/lib/python3.8/argparse.py", line 1858, in take_action
    argument_values = self._get_values

TypeError: object of type 'NoneType' has no len()